**Import libraries**

In [1642]:
import pandas as pd
import re 
from unidecode import unidecode
from pprint import pprint

from langdetect import detect
import openpyxl

**Inputs**

In [1643]:
file_name='fake_emails_utf8.csv'

replace=dict({
                '\n': ' '
             })

**Load data**

In [1644]:
data = pd.read_csv('input/' + file_name, encoding='utf-8', delimiter=';').replace(replace, regex=True)
data.head(30)

,email
0,"Dobrý den, chtěla bych se zeptat, jaký je akt..."
1,"Pěkný den, rád bych se dozvěděl více o aktuál..."
2,"Dobrý den, mohl byste mi, prosím, poskytnout ..."
3,Moje zásilka byla doručena na jiné místo než j...
4,Doručili jste zásilku na jiné místo. Prosím po...
5,Zdravím Kde je moje zásilka č.AB123456CD ?????...
6,"Pěkný večer, Kde je prosím moje zásilka s kóde..."
7,"Dobrý den, Kde je moje zásilka? Navštívil jse..."
8,"Zdravím, Nevíte kde je moje zásilka? ****odpov..."
9,Vaše služby si strčte do prdele. Budu si stěžo...


In [1645]:
data.dtypes

email    object
dtype: object

## Cleaning data

**Convert items into string**

In [1646]:
# Convert all items to strings
sentences = [str(item) for item in data]

**Put data into a list**

In [1647]:
sentences = [item for sublist in data.values.tolist() for item in sublist]

sentences[:]

['Dobrý den,  chtěla bych se zeptat, jaký je aktuální stav doručení mé objednávky s číslem GH123456CD. Mohl byste mi poskytnout informace o aktuální poloze zásilky a odhadovaném termínu doručení?   Děkuji za odpověď.  Alena Nováková ',
 'Pěkný den,  rád bych se dozvěděl více o aktuálním stavu mé zásilky s číslem AB123499CD. Kde se právě nachází a kdy mohu očekávat doručení?  Děkuji za rychlou odpověď.  Předem děkuji,  Miloš Pekárek',
 'Dobrý den,  mohl byste mi, prosím, poskytnout podrobnosti o doručení mé objednávky číslo AB128856CD? Zajímá mě aktuální poloha zásilky a očekávaný termín dodání. https://www.novinky.cz/  Děkuji za vaši asistenci.  Pěkný den,  Petr Pekař',
 'Moje zásilka byla doručena na jiné místo než jsem chtěl. martin@martin ----odpověď----- Omlouváme se.',
 'Doručili jste zásilku na jiné místo. Prosím pošlete na adresu Alena Šestá, Praha 6. Alena Černá',
 'Zdravím Kde je moje zásilka č.AB123456CD ?????????? https://www.google.com/ Alena Oranžová',
 'Pěkný večer, Kde j

**Replace brackets <>**

In [1648]:
def replace_brackets(sentences):
    # Apply the pattern to replace clusters enclosed by < and > with an empty string in each sentence
    result_sentences = [re.sub(r'<[^>]*>', '', sentence) for sentence in sentences]

    return result_sentences

In [1649]:
cleaned = replace_brackets(sentences)
cleaned[:30]

['Dobrý den,  chtěla bych se zeptat, jaký je aktuální stav doručení mé objednávky s číslem GH123456CD. Mohl byste mi poskytnout informace o aktuální poloze zásilky a odhadovaném termínu doručení?   Děkuji za odpověď.  Alena Nováková ',
 'Pěkný den,  rád bych se dozvěděl více o aktuálním stavu mé zásilky s číslem AB123499CD. Kde se právě nachází a kdy mohu očekávat doručení?  Děkuji za rychlou odpověď.  Předem děkuji,  Miloš Pekárek',
 'Dobrý den,  mohl byste mi, prosím, poskytnout podrobnosti o doručení mé objednávky číslo AB128856CD? Zajímá mě aktuální poloha zásilky a očekávaný termín dodání. https://www.novinky.cz/  Děkuji za vaši asistenci.  Pěkný den,  Petr Pekař',
 'Moje zásilka byla doručena na jiné místo než jsem chtěl. martin@martin ----odpověď----- Omlouváme se.',
 'Doručili jste zásilku na jiné místo. Prosím pošlete na adresu Alena Šestá, Praha 6. Alena Černá',
 'Zdravím Kde je moje zásilka č.AB123456CD ?????????? https://www.google.com/ Alena Oranžová',
 'Pěkný večer, Kde j

**Replace parcel number**

In [1650]:
def replace_parcel_number(sentences):
    # Apply the pattern to replace the entire cluster with 'id' in each sentence
    result_sentences = [re.sub(r'([A-Z]+\d+[A-Z]+)', r'id', sentence) for sentence in sentences]

    return result_sentences

In [1651]:
cleaned=replace_parcel_number(cleaned)
cleaned[:]

['Dobrý den,  chtěla bych se zeptat, jaký je aktuální stav doručení mé objednávky s číslem id. Mohl byste mi poskytnout informace o aktuální poloze zásilky a odhadovaném termínu doručení?   Děkuji za odpověď.  Alena Nováková ',
 'Pěkný den,  rád bych se dozvěděl více o aktuálním stavu mé zásilky s číslem id. Kde se právě nachází a kdy mohu očekávat doručení?  Děkuji za rychlou odpověď.  Předem děkuji,  Miloš Pekárek',
 'Dobrý den,  mohl byste mi, prosím, poskytnout podrobnosti o doručení mé objednávky číslo id? Zajímá mě aktuální poloha zásilky a očekávaný termín dodání. https://www.novinky.cz/  Děkuji za vaši asistenci.  Pěkný den,  Petr Pekař',
 'Moje zásilka byla doručena na jiné místo než jsem chtěl. martin@martin ----odpověď----- Omlouváme se.',
 'Doručili jste zásilku na jiné místo. Prosím pošlete na adresu Alena Šestá, Praha 6. Alena Černá',
 'Zdravím Kde je moje zásilka č.id ?????????? https://www.google.com/ Alena Oranžová',
 'Pěkný večer, Kde je prosím moje zásilka s kódem as

**Delte after certain words**

In [1652]:
def delete_after_certain_word(sentences, patterns):
    # Check for defined patterns and merge them
    combined_pattern = '|'.join(map(re.escape, patterns))

    # Apply the function
    pattern = re.compile(f'({combined_pattern}).*', re.DOTALL)

    # Apply the pattern to each sentence in the list
    result_sentences = [re.sub(pattern, '', sentence) for sentence in sentences]

    return result_sentences

In [1653]:
patterns=['--',
          'Děkuji za',
          'Předem děkuji',
          'S přáním',
          'Odesláno z'
          ]

cleaned = delete_after_certain_word(cleaned, patterns)
cleaned[:]

['Dobrý den,  chtěla bych se zeptat, jaký je aktuální stav doručení mé objednávky s číslem id. Mohl byste mi poskytnout informace o aktuální poloze zásilky a odhadovaném termínu doručení?   ',
 'Pěkný den,  rád bych se dozvěděl více o aktuálním stavu mé zásilky s číslem id. Kde se právě nachází a kdy mohu očekávat doručení?  ',
 'Dobrý den,  mohl byste mi, prosím, poskytnout podrobnosti o doručení mé objednávky číslo id? Zajímá mě aktuální poloha zásilky a očekávaný termín dodání. https://www.novinky.cz/  ',
 'Moje zásilka byla doručena na jiné místo než jsem chtěl. martin@martin ',
 'Doručili jste zásilku na jiné místo. Prosím pošlete na adresu Alena Šestá, Praha 6. Alena Černá',
 'Zdravím Kde je moje zásilka č.id ?????????? https://www.google.com/ Alena Oranžová',
 'Pěkný večer, Kde je prosím moje zásilka s kódem as21g5z99 Alena Červená Bílá',
 'Dobrý den, Kde je moje zásilka?  Navštívil jsem stránky www.zasilka.com ale nic jsem nezjistil? Kontaktujte mě alena@cervena.com, adresa: Vr

**Heading-remove certain words**

In [1654]:
def delete_certain_words(sentences, patterns):
    # Check for defined patterns and merge them
    combined_pattern = '|'.join(map(re.escape, patterns))

    # Apply the function
    pattern = re.compile(f'({combined_pattern})(\\S*|$)')

    # Apply the pattern to each sentence in the list
    result_sentences = [re.sub(pattern, '', sentence) for sentence in sentences]

    return result_sentences

In [1655]:
patterns=['Dobrý den',
          'Dobrý večer',
          'Pěkný večer',
          'Zdravím'
          ]
cleaned = delete_certain_words(cleaned, patterns)
cleaned[:]

['  chtěla bych se zeptat, jaký je aktuální stav doručení mé objednávky s číslem id. Mohl byste mi poskytnout informace o aktuální poloze zásilky a odhadovaném termínu doručení?   ',
 'Pěkný den,  rád bych se dozvěděl více o aktuálním stavu mé zásilky s číslem id. Kde se právě nachází a kdy mohu očekávat doručení?  ',
 '  mohl byste mi, prosím, poskytnout podrobnosti o doručení mé objednávky číslo id? Zajímá mě aktuální poloha zásilky a očekávaný termín dodání. https://www.novinky.cz/  ',
 'Moje zásilka byla doručena na jiné místo než jsem chtěl. martin@martin ',
 'Doručili jste zásilku na jiné místo. Prosím pošlete na adresu Alena Šestá, Praha 6. Alena Černá',
 ' Kde je moje zásilka č.id ?????????? https://www.google.com/ Alena Oranžová',
 ' Kde je prosím moje zásilka s kódem as21g5z99 Alena Červená Bílá',
 ' Kde je moje zásilka?  Navštívil jsem stránky www.zasilka.com ale nic jsem nezjistil? Kontaktujte mě alena@cervena.com, adresa: Vrchlabí 54, tel.789789789. ',
 ' Nevíte kde je moj

**Remove last 2 and 3 words if they start with capital letter**

In [1656]:
def remove_last_words_with_capital(sentences, min_num_words, max_num_words):
    # Apply the pattern to remove the last 2 or 3 words (or any range) starting with capital letters
    pattern = r'\b(?:[A-ZÁČĎÉĚÍŇÓŘŠŤÚŮÝŽ][^\s]*\s+){' + str(min_num_words - 1) + r',' + str(max_num_words - 1) + r'}[A-ZÁČĎÉĚÍŇÓŘŠŤÚŮÝŽ][^\s]*\s*$'
    result_sentences = [re.sub(pattern, '', sentence) for sentence in sentences]

    return result_sentences

In [1657]:
cleaned=remove_last_words_with_capital(cleaned, min_num_words=2, max_num_words=3)
cleaned[:]

['  chtěla bych se zeptat, jaký je aktuální stav doručení mé objednávky s číslem id. Mohl byste mi poskytnout informace o aktuální poloze zásilky a odhadovaném termínu doručení?   ',
 'Pěkný den,  rád bych se dozvěděl více o aktuálním stavu mé zásilky s číslem id. Kde se právě nachází a kdy mohu očekávat doručení?  ',
 '  mohl byste mi, prosím, poskytnout podrobnosti o doručení mé objednávky číslo id? Zajímá mě aktuální poloha zásilky a očekávaný termín dodání. https://www.novinky.cz/  ',
 'Moje zásilka byla doručena na jiné místo než jsem chtěl. martin@martin ',
 'Doručili jste zásilku na jiné místo. Prosím pošlete na adresu Alena Šestá, Praha 6. ',
 ' Kde je moje zásilka č.id ?????????? https://www.google.com/ ',
 ' Kde je prosím moje zásilka s kódem as21g5z99 ',
 ' Kde je moje zásilka?  Navštívil jsem stránky www.zasilka.com ale nic jsem nezjistil? Kontaktujte mě alena@cervena.com, adresa: Vrchlabí 54, tel.789789789. ',
 ' Nevíte kde je moje zásilka? ****odpověď**** ',
 'Vaše služby

In [1658]:
def remove_email(sentences):
    # Define the patterns to match email addresses
    var1 = r'\w+@\w+\.\w+'
    var2 = r'(\w+)@(\w+\-\w+\.\w+)'
    var3 = r'\w+@\w+'    
    #var4 = r'@\w+\.\w+' #does not work!!!!!!!!!!

    pattern = re.compile(fr'\b({var1}|{var2}|{var3})\b')
   
    # Apply the pattern to each sentence in the list
    result_sentences = [re.sub(pattern, 'email', sentence) for sentence in sentences]

    return result_sentences




""" # Example
sentences_to_remove_email = ['my email address is ahoj@ahoj ok', 
                             'Replace cluster@with.cz time', 
                             'Try this sona@bastl-instruments.com ok'
                             ]

# Call the function
result = remove_email(sentences_to_remove_email)
result """


" # Example\nsentences_to_remove_email = ['my email address is ahoj@ahoj ok', \n                             'Replace cluster@with.cz time', \n                             'Try this sona@bastl-instruments.com ok'\n                             ]\n\n# Call the function\nresult = remove_email(sentences_to_remove_email)\nresult "

In [1659]:
cleaned=remove_email(cleaned)
cleaned[:]

['  chtěla bych se zeptat, jaký je aktuální stav doručení mé objednávky s číslem id. Mohl byste mi poskytnout informace o aktuální poloze zásilky a odhadovaném termínu doručení?   ',
 'Pěkný den,  rád bych se dozvěděl více o aktuálním stavu mé zásilky s číslem id. Kde se právě nachází a kdy mohu očekávat doručení?  ',
 '  mohl byste mi, prosím, poskytnout podrobnosti o doručení mé objednávky číslo id? Zajímá mě aktuální poloha zásilky a očekávaný termín dodání. https://www.novinky.cz/  ',
 'Moje zásilka byla doručena na jiné místo než jsem chtěl. email ',
 'Doručili jste zásilku na jiné místo. Prosím pošlete na adresu Alena Šestá, Praha 6. ',
 ' Kde je moje zásilka č.id ?????????? https://www.google.com/ ',
 ' Kde je prosím moje zásilka s kódem as21g5z99 ',
 ' Kde je moje zásilka?  Navštívil jsem stránky www.zasilka.com ale nic jsem nezjistil? Kontaktujte mě email, adresa: Vrchlabí 54, tel.789789789. ',
 ' Nevíte kde je moje zásilka? ****odpověď**** ',
 'Vaše služby si strčte do prdele

**Replace expresions starting with http and www**

In [1660]:
def starting_with_replace(sentences, var: str, repl: str):

    # Define the pattern to match expressions starting with www or http
    pattern = re.compile(fr'\b(?:{re.escape(var)})\S*\b', flags=re.IGNORECASE)

    # Apply the pattern to each sentence in the list
    result_sentences = [re.sub(pattern, repl, sentence) for sentence in sentences]

    return result_sentences



""" # Example:
sentences = [ "This sentence has no specific www.pokus.com",
             "Check out this link: http://example.com.",
             "This sentence has no specific www.pokus.cz"]


result1 = starting_with_replace(sentences, 'http', 'replaced1')
result2 = starting_with_replace(sentences, 'www', 'replaced2')
print(result1)
print(result2) """

' # Example:\nsentences = [ "This sentence has no specific www.pokus.com",\n             "Check out this link: http://example.com.",\n             "This sentence has no specific www.pokus.cz"]\n\n\nresult1 = starting_with_replace(sentences, \'http\', \'replaced1\')\nresult2 = starting_with_replace(sentences, \'www\', \'replaced2\')\nprint(result1)\nprint(result2) '

In [1661]:
cleaned = starting_with_replace(cleaned, 'www', 'website')
cleaned = starting_with_replace(cleaned, 'http', 'website')
cleaned[:]

['  chtěla bych se zeptat, jaký je aktuální stav doručení mé objednávky s číslem id. Mohl byste mi poskytnout informace o aktuální poloze zásilky a odhadovaném termínu doručení?   ',
 'Pěkný den,  rád bych se dozvěděl více o aktuálním stavu mé zásilky s číslem id. Kde se právě nachází a kdy mohu očekávat doručení?  ',
 '  mohl byste mi, prosím, poskytnout podrobnosti o doručení mé objednávky číslo id? Zajímá mě aktuální poloha zásilky a očekávaný termín dodání. website/  ',
 'Moje zásilka byla doručena na jiné místo než jsem chtěl. email ',
 'Doručili jste zásilku na jiné místo. Prosím pošlete na adresu Alena Šestá, Praha 6. ',
 ' Kde je moje zásilka č.id ?????????? website/ ',
 ' Kde je prosím moje zásilka s kódem as21g5z99 ',
 ' Kde je moje zásilka?  Navštívil jsem stránky website ale nic jsem nezjistil? Kontaktujte mě email, adresa: Vrchlabí 54, tel.789789789. ',
 ' Nevíte kde je moje zásilka? ****odpověď**** ',
 'Vaše služby si strčte do prdele. Budu si stěžovat!!!! ',
 'Vaše služb

**Replace date, time and phone number**

In [1662]:
def replace_number_with_date(sentences):

    # define long dates dd.mm.yyy
    var1=r'\d{1,2}\.\d{1,2}\.\d{1,4}'
    var2=r'\d{1,2}\.\s\d{1,2}\.\s\d{1,4}'

    # define short dates dd.mm
    var3=r'\d{1,2}\.\d{1,2}'
    var4=r'\d{1,2}\.\s\d{1,2}'

    pattern = re.compile(fr'\b({var1}|{var2}|{var3}|{var4})\b')

    # Define the replacement string
    repl = 'konkrétní datum'

    # Apply the pattern to each sentence in the list
    result_sentences = [re.sub(pattern, repl, sentence) for sentence in sentences]

    return result_sentences

In [1663]:
def replace_number_with_time(sentences):

    # replace time
    var1=r'\d{1,2}\:\d{1,2}'
    pattern = re.compile(fr'\b({var1})\b')

    # Define the replacement string
    repl = 'konkrétní čas'

    # Apply the pattern to each sentence in the list
    result_sentences = [re.sub(pattern, repl, sentence) for sentence in sentences]

    return result_sentences

In [1664]:
def replace_number_with_telephone(sentences):

    # replace time
    var1=r'\btel\.\d+\b'
    pattern = re.compile(fr'{var1}')

    # Define the replacement string
    repl = 'telefonní číslo'

    # Apply the pattern to each sentence in the list
    result_sentences = [re.sub(pattern, repl, sentence) for sentence in sentences]

    return result_sentences


""" # Example usage:
sentences = ["Visit tel.5465465465 for more information.",
             "this date 20.12.2023 is nice",
             "this date 20. 12. 2023 is awsome",
             "this date 20. 12. is awful",
             "this date 20.12. is bad",
             "this time 20:12 is not good"
             ]

result = replace_number_with_telephone(sentences)
result = replace_number_with_date(result)
result = replace_number_with_time(result)
pprint(result) """

' # Example usage:\nsentences = ["Visit tel.5465465465 for more information.",\n             "this date 20.12.2023 is nice",\n             "this date 20. 12. 2023 is awsome",\n             "this date 20. 12. is awful",\n             "this date 20.12. is bad",\n             "this time 20:12 is not good"\n             ]\n\nresult = replace_number_with_telephone(sentences)\nresult = replace_number_with_date(result)\nresult = replace_number_with_time(result)\npprint(result) '

In [1665]:
cleaned = replace_number_with_telephone(cleaned)
cleaned = replace_number_with_date(cleaned)
cleaned = replace_number_with_time(cleaned)
cleaned[:5]

['  chtěla bych se zeptat, jaký je aktuální stav doručení mé objednávky s číslem id. Mohl byste mi poskytnout informace o aktuální poloze zásilky a odhadovaném termínu doručení?   ',
 'Pěkný den,  rád bych se dozvěděl více o aktuálním stavu mé zásilky s číslem id. Kde se právě nachází a kdy mohu očekávat doručení?  ',
 '  mohl byste mi, prosím, poskytnout podrobnosti o doručení mé objednávky číslo id? Zajímá mě aktuální poloha zásilky a očekávaný termín dodání. website/  ',
 'Moje zásilka byla doručena na jiné místo než jsem chtěl. email ',
 'Doručili jste zásilku na jiné místo. Prosím pošlete na adresu Alena Šestá, Praha 6. ']

**Delete alphanumeric**

In [1666]:
def remove_alphanumeric_words(sentences):
    # Pattern to match words containing both letters and numbers
    pattern = r'\b(?:[A-Za-zÁ-Žá-ž]+\d+|\d+[A-Za-zÁ-Žá-ž]+)[A-Za-zÁ-Žá-ž\d]*\b'
    result_sentences = [re.sub(pattern, '', sentence) for sentence in sentences]

    return result_sentences

""" # Example usage:
sentences = ["Visit s5s5s5ssfbvbbh for more information.",
             "Check out this 555ggg6dddd",
             "This sentence has no specific expression."]


result = remove_alphanumeric_words(sentences)
pprint(result) """

' # Example usage:\nsentences = ["Visit s5s5s5ssfbvbbh for more information.",\n             "Check out this 555ggg6dddd",\n             "This sentence has no specific expression."]\n\n\nresult = remove_alphanumeric_words(sentences)\npprint(result) '

In [1667]:
cleaned = remove_alphanumeric_words(cleaned)
cleaned[:5]

['  chtěla bych se zeptat, jaký je aktuální stav doručení mé objednávky s číslem id. Mohl byste mi poskytnout informace o aktuální poloze zásilky a odhadovaném termínu doručení?   ',
 'Pěkný den,  rád bych se dozvěděl více o aktuálním stavu mé zásilky s číslem id. Kde se právě nachází a kdy mohu očekávat doručení?  ',
 '  mohl byste mi, prosím, poskytnout podrobnosti o doručení mé objednávky číslo id? Zajímá mě aktuální poloha zásilky a očekávaný termín dodání. website/  ',
 'Moje zásilka byla doručena na jiné místo než jsem chtěl. email ',
 'Doručili jste zásilku na jiné místo. Prosím pošlete na adresu Alena Šestá, Praha 6. ']

**Replace numbers with words**

In [1668]:
def replace_numbers_with_words(sentences):
    # Define the mapping of digits to words
    digit_to_word = {'1': 'jeden', '2': 'dva', '3': 'tři', '4': 'čtyři', '5': 'pět'}

    # Create a pattern to match any of the specified digits
    pattern = r'\b([1-5])\b'
    
    # Replace each digit with its corresponding word
    result_sentences = [re.sub(pattern, lambda x: digit_to_word[x.group()], sentence) for sentence in sentences]

    return result_sentences


""" # Example usage:
sentences = ["This is an example with 1 and 3 numbers.", "Another sentence with česká diakritika and 5 special characters!"]

result = replace_numbers_with_words(sentences)
pprint(result) """

' # Example usage:\nsentences = ["This is an example with 1 and 3 numbers.", "Another sentence with česká diakritika and 5 special characters!"]\n\nresult = replace_numbers_with_words(sentences)\npprint(result) '

In [1669]:
cleaned = replace_numbers_with_words(cleaned)
cleaned[:5]

['  chtěla bych se zeptat, jaký je aktuální stav doručení mé objednávky s číslem id. Mohl byste mi poskytnout informace o aktuální poloze zásilky a odhadovaném termínu doručení?   ',
 'Pěkný den,  rád bych se dozvěděl více o aktuálním stavu mé zásilky s číslem id. Kde se právě nachází a kdy mohu očekávat doručení?  ',
 '  mohl byste mi, prosím, poskytnout podrobnosti o doručení mé objednávky číslo id? Zajímá mě aktuální poloha zásilky a očekávaný termín dodání. website/  ',
 'Moje zásilka byla doručena na jiné místo než jsem chtěl. email ',
 'Doručili jste zásilku na jiné místo. Prosím pošlete na adresu Alena Šestá, Praha 6. ']

**Delete standalone numbers**

In [1670]:
def remove_numeric_words(sentences):
    # Pattern to match standalone numbers
    pattern = r'\b\d+\b'
    result_sentences = [re.sub(pattern, '', sentence) for sentence in sentences]

    return result_sentences

""" # Example usage:s
sentences = ["This is an example with 123 numbers.", "Another sentence with abc123 and special characters!"]
result = remove_numeric_words(sentences)
pprint(result) """

' # Example usage:s\nsentences = ["This is an example with 123 numbers.", "Another sentence with abc123 and special characters!"]\nresult = remove_numeric_words(sentences)\npprint(result) '

In [1671]:
cleaned = remove_numeric_words(cleaned)
cleaned[:5]

['  chtěla bych se zeptat, jaký je aktuální stav doručení mé objednávky s číslem id. Mohl byste mi poskytnout informace o aktuální poloze zásilky a odhadovaném termínu doručení?   ',
 'Pěkný den,  rád bych se dozvěděl více o aktuálním stavu mé zásilky s číslem id. Kde se právě nachází a kdy mohu očekávat doručení?  ',
 '  mohl byste mi, prosím, poskytnout podrobnosti o doručení mé objednávky číslo id? Zajímá mě aktuální poloha zásilky a očekávaný termín dodání. website/  ',
 'Moje zásilka byla doručena na jiné místo než jsem chtěl. email ',
 'Doručili jste zásilku na jiné místo. Prosím pošlete na adresu Alena Šestá, Praha . ']

**Simplify repeating symbols**

In [1672]:
def replace_repeating_characters(sentences):
    # Apply the pattern to replace clusters of the same symbols with a single symbol in each sentence
    result_sentences = [re.sub(r'(\S)\1+', r'\1', sentence, flags=re.UNICODE) for sentence in sentences]

    return result_sentences


""" # Example usage:s
sentences = ["This is an example with 123 numbers!!!!!!!!!", 
            "Another sentence with abc123 and special characters??????",
            'hi how aaaaare are youuuuuu?????'
            ]

result = replace_repeating_characters(sentences)
pprint(result) """

' # Example usage:s\nsentences = ["This is an example with 123 numbers!!!!!!!!!", \n            "Another sentence with abc123 and special characters??????",\n            \'hi how aaaaare are youuuuuu?????\'\n            ]\n\nresult = replace_repeating_characters(sentences)\npprint(result) '

In [1673]:
cleaned = replace_repeating_characters(cleaned)
cleaned[:5]

['  chtěla bych se zeptat, jaký je aktuální stav doručení mé objednávky s číslem id. Mohl byste mi poskytnout informace o aktuální poloze zásilky a odhadovaném termínu doručení?   ',
 'Pěkný den,  rád bych se dozvěděl více o aktuálním stavu mé zásilky s číslem id. Kde se právě nachází a kdy mohu očekávat doručení?  ',
 '  mohl byste mi, prosím, poskytnout podrobnosti o doručení mé objednávky číslo id? Zajímá mě aktuální poloha zásilky a očekávaný termín dodání. website/  ',
 'Moje zásilka byla doručena na jiné místo než jsem chtěl. email ',
 'Doručili jste zásilku na jiné místo. Prosím pošlete na adresu Alena Šestá, Praha . ']

**Delete weird punctuation**

In [1674]:
punct_list='[],]\/()|:"+*©@'

def remove_punctuation(sentences):
    processed_sentences = [re.sub(f'[{re.escape(punct_list)}]', ' ', sentence) for sentence in sentences]
    return processed_sentences


""" # example
sentences= ['ahoj@majk, dfqdas', 'fadsf/ssss', 'asdas:asdas()d']
result=remove_punctuation(sentences)
pprint(result) """

" # example\nsentences= ['ahoj@majk, dfqdas', 'fadsf/ssss', 'asdas:asdas()d']\nresult=remove_punctuation(sentences)\npprint(result) "

In [1675]:
cleaned = remove_punctuation(cleaned)
cleaned[:5]

['  chtěla bych se zeptat  jaký je aktuální stav doručení mé objednávky s číslem id. Mohl byste mi poskytnout informace o aktuální poloze zásilky a odhadovaném termínu doručení?   ',
 'Pěkný den   rád bych se dozvěděl více o aktuálním stavu mé zásilky s číslem id. Kde se právě nachází a kdy mohu očekávat doručení?  ',
 '  mohl byste mi  prosím  poskytnout podrobnosti o doručení mé objednávky číslo id? Zajímá mě aktuální poloha zásilky a očekávaný termín dodání. website   ',
 'Moje zásilka byla doručena na jiné místo než jsem chtěl. email ',
 'Doručili jste zásilku na jiné místo. Prosím pošlete na adresu Alena Šestá  Praha . ']

**Names**

In [1676]:
# check diacritics
def has_diacritics(name):
    return any(char.isalpha() and char != unidecode(char) for char in name)

# add to the list
def add_names_without_diacritics(czech_names: list) -> list:
    [czech_names.append(unidecode(name)) if has_diacritics(name) else next for name in czech_names]
    return czech_names

In [1677]:
# read the file
czech_names = pd.read_csv('input/czech_names.csv', encoding='utf-8')
czech_names=czech_names['jméno'].tolist()
czech_names[:5]

['Adéla', 'Adriana', 'Agáta', 'Albína', 'Alena']

In [1678]:
# apply the function
czech_names=add_names_without_diacritics(czech_names)
czech_names[-10:]

['Vladimir',
 'Vojtech',
 'Zaboj',
 'Zavis',
 'Zbyhnev',
 'Zbynek',
 'Zbysek',
 'Zdenek',
 'Zoltan',
 'Zelimir']

In [1679]:
def convert_to_lowercase_and_keep_original(words):
    result_list = []
    for word in words:
        result_list.append(word)
        if word[0].isupper():
            result_list.append(word[0].lower() + word[1:])
    return result_list

""" # Example usage
word_list = ['Praha', 'praha', 'Brno', 'brno']
modified_list = convert_to_lowercase_and_keep_original(word_list)

pprint(modified_list) """

" # Example usage\nword_list = ['Praha', 'praha', 'Brno', 'brno']\nmodified_list = convert_to_lowercase_and_keep_original(word_list)\n\npprint(modified_list) "

In [1680]:
def delete_patterns(sentences, patterns):
    # Check for defined patterns and merge them
    combined_pattern = '|'.join(map(re.escape, patterns))

    # Apply the function
    pattern = re.compile(fr'\b({combined_pattern})\b')

    # Apply the pattern to each sentence in the list
    result_sentences = [re.sub(pattern, '', sentence) for sentence in sentences]

    return result_sentences


""" # example
sentences= ['ahoj Miloš jak sa mate', 
            'nevim o Milošovi vubec nic'
            ]

result=delete_patterns(sentences, czech_names)
pprint(result) """

" # example\nsentences= ['ahoj Miloš jak sa mate', \n            'nevim o Milošovi vubec nic'\n            ]\n\nresult=delete_patterns(sentences, czech_names)\npprint(result) "

In [1681]:
czech_names = convert_to_lowercase_and_keep_original(czech_names)
cleaned = delete_patterns(cleaned, czech_names)
cleaned[:5]

['  chtěla bych se zeptat  jaký je aktuální stav doručení mé objednávky s číslem id. Mohl byste mi poskytnout informace o aktuální poloze zásilky a odhadovaném termínu doručení?   ',
 'Pěkný den   rád bych se dozvěděl více o aktuálním stavu mé zásilky s číslem id. Kde se právě nachází a kdy mohu očekávat doručení?  ',
 '  mohl byste mi  prosím  poskytnout podrobnosti o doručení mé objednávky číslo id? Zajímá mě aktuální poloha zásilky a očekávaný termín dodání. website   ',
 'Moje zásilka byla doručena na jiné místo než jsem chtěl. email ',
 'Doručili jste zásilku na jiné místo. Prosím pošlete na adresu  Šestá  Praha . ']

**Cities**

In [1682]:
# read the file
czech_cities = pd.read_csv('input/czech_cities.csv', encoding='utf-8')
czech_cities=czech_cities['město'].tolist()

# Split each item in the original list and flatten the resulting list
czech_cities_split = [item.strip() for sublist in [item.split('•') for item in czech_cities] for item in sublist]

pprint(czech_cities_split[:5])

['Benešov', 'Bernartice', 'Bílkovice', 'Blažejovice', 'Borovnice']


In [1683]:
# apply function created before
czech_cities_split=add_names_without_diacritics(czech_cities_split)
len(czech_cities_split)

10626

In [1684]:
""" def convert_to_lowercase_and_keep_original(words):
    result_list = []
    for word in words:
        result_list.append(word)
        if word[0].isupper():
            result_list.append(word[0].lower() + word[1:])
    return result_list """

""" # Example usage
word_list = ['Praha', 'praha', 'Brno', 'brno']
modified_list = convert_to_lowercase_and_keep_original(word_list)

pprint(modified_list) """

" # Example usage\nword_list = ['Praha', 'praha', 'Brno', 'brno']\nmodified_list = convert_to_lowercase_and_keep_original(word_list)\n\npprint(modified_list) "

In [1685]:
""" czech_cities_split=convert_to_lowercase_and_keep_original(czech_cities_split)
len(czech_cities_split) """

' czech_cities_split=convert_to_lowercase_and_keep_original(czech_cities_split)\nlen(czech_cities_split) '

In [1686]:
# apply function created before
cleaned=delete_patterns(cleaned, czech_cities_split)
cleaned[:5]

['  chtěla bych se zeptat  jaký je aktuální stav doručení mé objednávky s číslem id. Mohl byste mi poskytnout informace o aktuální poloze zásilky a odhadovaném termínu doručení?   ',
 'Pěkný den   rád bych se dozvěděl více o aktuálním stavu mé zásilky s číslem id. Kde se právě nachází a kdy mohu očekávat doručení?  ',
 '  mohl byste mi  prosím  poskytnout podrobnosti o doručení mé objednávky číslo id? Zajímá mě aktuální poloha zásilky a očekávaný termín dodání. website   ',
 'Moje zásilka byla doručena na jiné místo než jsem chtěl. email ',
 'Doručili jste zásilku na jiné místo. Prosím pošlete na adresu  Šestá   . ']

**Lowercase**

In [1687]:
def change_lowercase(list):
    processed_sentences = [sentence.lower() for sentence in list]
    return processed_sentences

""" # Example usage
word_list = ['Praha', 'praha', 'Brno', 'brno']
modified_list = change_lowercase(word_list)

pprint(modified_list) """

" # Example usage\nword_list = ['Praha', 'praha', 'Brno', 'brno']\nmodified_list = change_lowercase(word_list)\n\npprint(modified_list) "

In [1688]:
cleaned = change_lowercase(cleaned)
cleaned[:5]

['  chtěla bych se zeptat  jaký je aktuální stav doručení mé objednávky s číslem id. mohl byste mi poskytnout informace o aktuální poloze zásilky a odhadovaném termínu doručení?   ',
 'pěkný den   rád bych se dozvěděl více o aktuálním stavu mé zásilky s číslem id. kde se právě nachází a kdy mohu očekávat doručení?  ',
 '  mohl byste mi  prosím  poskytnout podrobnosti o doručení mé objednávky číslo id? zajímá mě aktuální poloha zásilky a očekávaný termín dodání. website   ',
 'moje zásilka byla doručena na jiné místo než jsem chtěl. email ',
 'doručili jste zásilku na jiné místo. prosím pošlete na adresu  šestá   . ']

## Filter the whole sentences

**Delete short or empty sentences**

In [1689]:
def remove_short_sentences(sentences, max_words=1):
    # Filter out sentences with the number of words less than or equal to max_words
    result_sentences = [sentence for sentence in sentences if len(sentence.split()) > max_words]

    return result_sentences

""" # Example usage:
sentences = ["Hodně slov v této větě", 
             "jedno", 
             'tři slova tady',
             'dvě slova',
             ''
             ]

result = remove_short_sentences(sentences, max_words=2)
pprint(result) """

' # Example usage:\nsentences = ["Hodně slov v této větě", \n             "jedno", \n             \'tři slova tady\',\n             \'dvě slova\',\n             \'\'\n             ]\n\nresult = remove_short_sentences(sentences, max_words=2)\npprint(result) '

In [1690]:
before=len(cleaned)

cleaned = remove_short_sentences(cleaned)
cleaned[:5]

after=len(cleaned)

print(f' before: {before} \n after: {after} \n delta: {before-after}')

 before: 28 
 after: 23 
 delta: 5


**Detect language and keep just Czech**

In [1691]:
def filter_czech_slovak_sentences(sentences):
    def detect_language(text):
        try:
            language = detect(text)
            return language
        except:
            return "Unknown"
    refused=[]
    return [sentence for sentence in sentences if detect_language(sentence) in ['cs']]



""" # Example sentences in multiple languages
sentences = [
    "Hello, how are you?",
    "Bonjour, comment ça va?",
    "Hola, ¿cómo estás?",
    "Ahoj, jak se máš?",
    'Máš se dobre večer?',
    "Ako sa máš?",
    'čau příště',
    "Привет, как дела?",
]

# Call the function to filter Czech and Slovak sentences
czech_slovak_sentences = filter_czech_slovak_sentences(sentences)

pprint(czech_slovak_sentences) """

' # Example sentences in multiple languages\nsentences = [\n    "Hello, how are you?",\n    "Bonjour, comment ça va?",\n    "Hola, ¿cómo estás?",\n    "Ahoj, jak se máš?",\n    \'Máš se dobre večer?\',\n    "Ako sa máš?",\n    \'čau příště\',\n    "Привет, как дела?",\n]\n\n# Call the function to filter Czech and Slovak sentences\nczech_slovak_sentences = filter_czech_slovak_sentences(sentences)\n\npprint(czech_slovak_sentences) '

In [1692]:
before=len(cleaned)

cleaned = filter_czech_slovak_sentences(cleaned)
cleaned[:5]

after=len(cleaned)

print(f' before: {before} \n after: {after} \n delta: {before-after}')

 before: 23 
 after: 20 
 delta: 3


**Remove duplicates**

In [1693]:
def remove_duplicates(sentences):
    unique_sentences = set()
    filtered_sentences = [sentence for sentence in sentences if not (sentence in unique_sentences or unique_sentences.add(sentence))]
    return filtered_sentences


""" # Example usage:
sentences = [
    "This is a sentence.",
    "Another sentence here.",
    "This is a sentence.",  # Duplicate
    "Yet another unique sentence.",
    "Another sentence here.",  # Duplicate
    "One more unique sentence."
]

result = remove_duplicates(sentences)
pprint(result) """

' # Example usage:\nsentences = [\n    "This is a sentence.",\n    "Another sentence here.",\n    "This is a sentence.",  # Duplicate\n    "Yet another unique sentence.",\n    "Another sentence here.",  # Duplicate\n    "One more unique sentence."\n]\n\nresult = remove_duplicates(sentences)\npprint(result) '

In [1694]:
before=len(cleaned)

cleaned = remove_duplicates(cleaned)
cleaned[:20]

after=len(cleaned)

print(f' before: {before} \n after: {after} \n delta: {before-after}')

 before: 20 
 after: 19 
 delta: 1


## Export the results

In [1695]:
results = pd.DataFrame({'sentences': cleaned})
results.head(5)

,sentences
0,chtěla bych se zeptat jaký je aktuální stav...
1,pěkný den rád bych se dozvěděl více o aktuál...
2,mohl byste mi prosím poskytnout podrobnost...
3,moje zásilka byla doručena na jiné místo než j...
4,doručili jste zásilku na jiné místo. prosím po...


In [1696]:
results.to_excel('output/results.xlsx', index=False)